<a href="https://colab.research.google.com/github/manish2021iitd/Deep-Learning/blob/main/DLassignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Q1
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist

# Load Fashion-MNIST dataset
(training_images, training_labels), (testing_images,testing_labels) = fashion_mnist.load_data()

# Plot one sample image for each class
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

plt.figure(figsize=(8, 3))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(training_images[training_labels == i][0], cmap='gray')
    plt.title(class_names[i])
    plt.axis('off')
plt.show()


In [ ]:
#Q2
import numpy as np

class NeuralNetwork:
    def __init__(self, input_size, hidden_sizes, output_size):
        self.input_size = input_size #number of inputs nodes
        self.hidden_sizes = hidden_sizes #list of neurons in each hidden layer
        self.output_size = output_size #number of output neurons in output layer
        self.weights = [] #list of weight matrices for each layer
        self.biases = []  #list of bias vectors for each hidden layer and output layer

        #innitializing the weights and biases for each layer
        sizes = [input_size] + hidden_sizes + [output_size] #using list concatination to make a list of number of nodes in each layer
        for i in range(len(sizes) - 1):
            self.weights.append(np.random.randn(sizes[i+1], sizes[i]))
            self.biases.append(np.random.randn(sizes[i+1]))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def softmax(self, x):
        exp_scores = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    def forward(self, X):
      # Forward pass
      activations = [X]
      for i in range(len(self.weights)):
        z = np.dot(activations[-1], self.weights[i].T) + self.biases[i]  # Transpose weight matrix
        a = self.sigmoid(z) if i < len(self.weights) - 1 else self.softmax(z)
        activations.append(a)
      return activations

    def predict(self, X):
        #predict output probabilities
        activations = self.forward(X)
        return activations[-1]

#load fashion-mnist dataset
from keras.datasets import fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

#preprocess the data
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

#define neural network
input_size = X_train.shape[1]  #28x28 images flattened
hidden_sizes = [128, 64]  #size of hidden layers
output_size = 10  # 10 classes in fashion-mnist

#initialize neural network
model = NeuralNetwork(input_size, hidden_sizes, output_size)

#predict probabilities for the test set
probabilities = model.predict(X_test)

#print the output probabilities
print("Output Probabilities:", probabilities)

In [ ]:
import numpy as np

class NeuralNetwork:
    def __init__(self, input_size, hidden_sizes, output_size):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.weights = []
        self.biases = []

        #initialize weights and biases for each layer
        sizes = [input_size] + hidden_sizes + [output_size]
        for i in range(len(sizes) - 1):
            self.weights.append(np.random.randn(sizes[i+1], sizes[i]))
            self.biases.append(np.random.randn(sizes[i+1],1))

        #initialize velocities for momentum-based optimization
        self.velocities = [np.zeros_like(w) for w in self.weights]

        #initialize rmsprop parameters
        self.rmsprop_cache_w = [np.zeros_like(w) for w in self.weights]
        self.rmsprop_cache_b = [np.zeros_like(b) for b in self.biases]

        #initialize adam parameters
        self.adam_m_w = [np.zeros_like(w) for w in self.weights]
        self.adam_m_b = [np.zeros_like(b) for b in self.biases]
        self.adam_v_w = [np.zeros_like(w) for w in self.weights]
        self.adam_v_b = [np.zeros_like(b) for b in self.biases]

    def sigmoid(self, x):
      return 1 / (1 + np.exp(-x))

    def d_sigmoid(self,x):
      return self.sigmoid(x)*(1-self.sigmoid(x))

    def softmax(self, x):
      exp_scores = np.exp(x - np.max(x, axis=1, keepdims=True))
      return exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    def logloss(self, y, y_label):
      error = -(y_label*np.log(y+1e-10)+(1-y_label)*np.log(1-y+1e-10))
      return np.sum(error, axis = 0)/error.shape[0]

    def Cross_entropy(self, y, y_hat):
      da = (1-y)/(1-y_hat) - (y)/(y_hat+1e-10)
      epoch_loss = self.__logloss(y_hat, y)
      return da, epoch_loss

    #forwardpropagation
    def forward(self, X):
      activations = [X] #create a list of all activations
      for i in range(len(self.weights)):
        z = np.dot(self.weights[i],activations[-1].T) + self.biases[i]  #preactivation
        a = self.sigmoid(z) if i < len(self.weights) - 1 else self.softmax(z) #activation/output
        activations.append(a) #appending the activations list
      return activations

    #backwardpropagation
    def backward(self, y, activations):
        delta = activations[-1] - y
        for i in range(len(self.weights) - 1, -1, -1):
            dz = delta * (activations[i+1] * (1 - activations[i+1])) if i < len(self.weights) - 1 else delta
            dw = np.dot(dz.T, activations[i])
            db = np.sum(dz, axis=0)
            delta = np.dot(dz, self.weights[i])
        return dw,db


    def update(self, X, y, optimizer, epochs, learning_rate=0.0001, beta1=0.9, beta2=0.999, epsilon=1e-8):
      for _ in range(epochs):
        activations = self.forward(X)
        dw, db = self.backward(y, activations)

        if optimizer == 'sgd':
          for i in range(len(self.weights)):
            self.weights[i] -= learning_rate * dw[i]
            self.biases[i] -= learning_rate * db[i]

        elif optimizer == 'momentum':
          for i in range(len(self.weights)):
            self.velocities[i] = beta1 * self.velocities[i] + (1 - beta1) * dw[i]
            self.weights[i] -= learning_rate * self.velocities[i]
            self.biases[i] -= learning_rate * db[i]

        elif optimizer == 'nesterov':
          lookahead_weights = [w - beta1 * v for w, v in zip(self.weights, self.velocities)]
          lookahead_biases = [b - beta1 * v[-1] for b, v in zip(self.biases, self.velocities)]
          lookahead_activations = self.forward(X)
          lookahead_dw, lookahead_db = self.backward(y, lookahead_activations)
          for i in range(len(self.weights)):
            self.velocities[i] = beta1 * self.velocities[i] + (1 - beta1) * lookahead_dw[i]
            self.weights[i] -= learning_rate * self.velocities[i]
            self.biases[i] -= learning_rate * lookahead_db[i]

        elif optimizer == 'rmsprop':
          for i in range(len(self.weights)):
            self.rmsprop_cache_w[i] = beta1 * self.rmsprop_cache_w[i] + (1 - beta1) * dw[i]**2
            self.rmsprop_cache_b[i] = beta1 * self.rmsprop_cache_b[i] + (1 - beta1) * db[i]**2
            self.weights[i] -= learning_rate * dw[i] / (np.sqrt(self.rmsprop_cache_w[i]) + epsilon)
            self.biases[i] -= learning_rate * db[i] / (np.sqrt(self.rmsprop_cache_b[i]) + epsilon)

        elif optimizer == 'adam':
          for i in range(len(self.weights)):
            self.adam_m_w[i] = beta1 * self.adam_m_w[i] + (1 - beta1) * dw[i]
            self.adam_m_b[i] = beta1 * self.adam_m_b[i] + (1 - beta1) * db[i]
            self.adam_v_w[i] = beta2 * self.adam_v_w[i] + (1 - beta2) * dw[i]**2
            self.adam_v_b[i] = beta2 * self.adam_v_b[i] + (1 - beta2) * db[i]**2
            m_w_hat = self.adam_m_w[i] / (1 - beta1**(i+1))
            m_b_hat = self.adam_m_b[i] / (1 - beta1**(i+1))
            v_w_hat = self.adam_v_w[i] / (1 - beta2**(i+1))
            v_b_hat = self.adam_v_b[i] / (1 - beta2**(i+1))
            self.weights[i] -= learning_rate * m_w_hat / (np.sqrt(v_w_hat) + epsilon)
            self.biases[i] -= learning_rate * m_b_hat / (np.sqrt(v_b_hat) + epsilon)

        elif optimizer == 'nadam':
          for i in range(len(self.weights)):
            self.adam_m_w[i] = beta1 * self.adam_m_w[i] + (1 - beta1) * dw[i]
            self.adam_m_b[i] = beta1 * self.adam_m_b[i] + (1 - beta1) * db[i]
            self.adam_v_w[i] = beta2 * self.adam_v_w[i] + (1 - beta2) * dw[i]**2
            self.adam_v_b[i] = beta2 * self.adam_v_b[i] + (1 - beta2) * db[i]**2
            m_w_hat = self.adam_m_w[i] / (1 - beta1**(i+1))
            m_b_hat = self.adam_m_b[i] / (1 - beta1**(i+1))
            v_w_hat = self.adam_v_w[i] / (1 - beta2**(i+1))
            v_b_hat = self.adam_v_b[i] / (1 - beta2**(i+1))
            self.weights[i] -= learning_rate * m_w_hat / (np.sqrt(v_w_hat) + epsilon)
            self.biases[i] -= learning_rate * m_b_hat / (np.sqrt(v_b_hat) + epsilon)


    def train(self, X, y, optimizer, epochs, batch_size):
      #train the neural network using different optimizers
      for epoch in range(epochs):
        for i in range(0, len(X), batch_size):
          X_batch = X[i:i+batch_size]
          y_batch = y[i:i+batch_size]
          self.update(X_batch,y_batch,optimizer,epochs)

    def test(self, X):
      activations = self.forward(X)
      return activations[-1]


#load fashion-mnist dataset
from keras.datasets import fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

#preprocess the data
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)
print(X_train.shape)
#define neural network
input_size = 784#size of inputs
hidden_sizes = [128, 64]  #size of neurons in each hidden layers
output_size = 10  #taking 10 output neurons as we have classify 10 classes in fashion-mnist

# Convert labels to one-hot encoding
num_classes = 10
y_train_one_hot = np.eye(num_classes)[y_train]
y_test_one_hot = np.eye(num_classes)[y_test]

#initialize neural network
model = NeuralNetwork(input_size, hidden_sizes, output_size)

#please choose optimizer = 'sgd'/'momentum'/'nesterov'/'rmsprop'/'adam'/'nadam'
#and number of epochs and batch size here
model.train(X_train, y_train_one_hot,'nadam', epochs=10, batch_size=32)

results = model.test(X_test)

results = np.argmax(results, axis=-1)
real_labels = np.argmax(y_test, axis=-1)
accuracy = sum(1*(results==real_labels))/len(results)
print("The accuracy of testing is ", accuracy*100, "%")
#References:
#1.https://cs229.stanford.edu/main_notes.pdf
#2.http://www.cse.iitm.ac.in/~miteshk/CS6910.html
#3.https://visualstudiomagazine.com/Articles/2017/06/01/Back-Propagation.aspx?Page=2
#4.https://medium.com/@ipylypenko/exploring-neural-networks-with-fashion-mnist-b0a8214b7b7b
#5.https://www.youtube.com/watch?v=Wo5dMEP_BbI&list=PLQVvvaa0QuDcjD5BAw2DxE6OF2tius3V3
#7.https://www.youtube.com/watch?v=LQvRhQwDOm0